In [1]:
!pip install ffmpeg-python

In [1]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import random
import os
from torch.nn import functional as F
import time
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Enhanced frame extraction with original resolution and color
def extract_frames(video_path, output_dir="frames"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Total frames: {frame_count}, FPS: {fps}, Resolution: {width}x{height}")
    
    frames = []
    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = f"{output_dir}/frame_{i:04d}.png"
        cv2.imwrite(frame_path, frame)
        frames.append((frame_path, (height, width)))
    cap.release()
    return frames, fps

# Specialized dataset for reflection removal
class ReflectionDataset(Dataset):
    def __init__(self, frame_data, patch_size=256, augment=True):
        self.frame_data = frame_data
        self.patch_size = patch_size
        self.augment = augment
        
    def __len__(self):
        return len(self.frame_data)
    
    def detect_reflections(self, frame):
        """
        More inclusive reflection detection for endoscopy images - 
        designed to catch a broader range of specular highlights
        """
        # Convert to multiple color spaces
        hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)

        # Extract channels
        s_channel = hsv[:,:,1]    # Saturation
        v_channel = hsv[:,:,2]    # Value/brightness
        l_channel = lab[:,:,0]    # Lightness

        # 1. Primary method: High brightness
        # More relaxed brightness threshold (top 5% instead of top 1.5%)
        v_thresh = np.percentile(v_channel, 95)  
        bright_areas = (v_channel > v_thresh).astype(np.uint8)

        # 2. Secondary method: High brightness with low saturation (classic specular)
        v_thresh_high = np.percentile(v_channel, 97)
        s_thresh_low = np.percentile(s_channel, 30)  # More inclusive saturation threshold
        specular_highlights = ((v_channel > v_thresh_high) & (s_channel < s_thresh_low)).astype(np.uint8)

        # 3. Third method: Local brightness peaks
        kernel_size = 9  # Smaller kernel to detect more localized brightness peaks
        max_filtered = cv2.dilate(gray, np.ones((kernel_size, kernel_size), np.uint8))
        local_maxima = ((gray == max_filtered) & (gray > np.percentile(gray, 92))).astype(np.uint8)

        # 4. LAB space extreme lightness
        l_thresh = np.percentile(l_channel, 95)
        lab_bright = (l_channel > l_thresh).astype(np.uint8)

        # Combine all methods
        combined_mask = cv2.bitwise_or(bright_areas, specular_highlights)
        combined_mask = cv2.bitwise_or(combined_mask, local_maxima)
        combined_mask = cv2.bitwise_or(combined_mask, lab_bright)

        # Morphological operations to connect nearby reflections
        kernel_close = np.ones((5, 5), np.uint8)
        kernel_open = np.ones((2, 2), np.uint8)  # Smaller kernel to preserve more details

        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel_close)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel_open)

        # Connected component filtering with smaller minimum size
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(cleaned_mask, 4)
        min_size = 2  # Smaller threshold to keep more reflection points
        refined_mask = np.zeros_like(cleaned_mask)

        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= min_size:
                refined_mask[labels == i] = 1

        # Dilate to ensure full coverage of reflections
        kernel_dilate = np.ones((3, 3), np.uint8)
        final_mask = cv2.dilate(refined_mask, kernel_dilate, iterations=1)

        return final_mask * 255  # Scale to 0-255 range
    

    
    def create_patch(self, image, mask):
        """Create a patch containing reflections for training"""
        h, w = image.shape[:2]
        
        # Find reflection areas
        y_indices, x_indices = np.where(mask > 0)
        
        if len(y_indices) > 0:
            # Choose a random reflection point
            idx = random.randrange(len(y_indices))
            center_y, center_x = y_indices[idx], x_indices[idx]
            
            # Calculate crop boundaries
            half_size = self.patch_size // 2
            x_start = max(0, min(center_x - half_size, w - self.patch_size))
            y_start = max(0, min(center_y - half_size, h - self.patch_size))
            
            # Extract patches
            image_patch = image[y_start:y_start+self.patch_size, x_start:x_start+self.patch_size].copy()
            mask_patch = mask[y_start:y_start+self.patch_size, x_start:x_start+self.patch_size].copy()
        else:
            # If no reflections, take a random patch
            x_start = random.randint(0, max(0, w - self.patch_size))
            y_start = random.randint(0, max(0, h - self.patch_size))
            image_patch = image[y_start:y_start+self.patch_size, x_start:x_start+self.patch_size].copy()
            mask_patch = mask[y_start:y_start+self.patch_size, x_start:x_start+self.patch_size].copy()
            
        return image_patch, mask_patch
    
    def augment_sample(self, image, mask):
        """Apply data augmentation"""
        # Random flip
        if random.random() > 0.5:
            image = np.fliplr(image).copy()
            mask = np.fliplr(mask).copy()
        
        # Random rotation
        k = random.randint(0, 3)
        if k > 0:
            image = np.rot90(image, k).copy()
            mask = np.rot90(mask, k).copy()
        
        return image, mask
    
    def __getitem__(self, idx):
        frame_path, _ = self.frame_data[idx]
        frame = cv2.imread(frame_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Resize for consistent processing
        frame = cv2.resize(frame, (512, 512))
        frame_float = frame.astype(np.float32) / 255.0
        
        # Detect reflections
        mask = self.detect_reflections(frame)
        mask_float = mask.astype(np.float32) / 255.0
        
        if self.augment:
            # Create patch for training
            image_patch, mask_patch = self.create_patch(frame_float, mask_float)
            image_patch, mask_patch = self.augment_sample(image_patch, mask_patch)
            
            # Convert to tensors
            image_tensor = torch.from_numpy(image_patch.transpose(2, 0, 1)).float()
            mask_tensor = torch.from_numpy(mask_patch).unsqueeze(0).float()
        else:
            # For inference, use the whole image
            image_tensor = torch.from_numpy(frame_float.transpose(2, 0, 1)).float()
            mask_tensor = torch.from_numpy(mask_float).unsqueeze(0).float()
        
        return image_tensor, mask_tensor

# Residual Block for Generator
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.InstanceNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.InstanceNorm2d(channels)
    
    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = self.relu(out)
        return out

# Simplified Generator
class InpaintingGenerator(nn.Module):
    def __init__(self):
        super(InpaintingGenerator, self).__init__()
        
        # Initial convolution - take image and mask as input
        self.init_conv = nn.Sequential(
            nn.Conv2d(4, 64, kernel_size=7, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        # Downsampling
        self.down1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.down2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        )
        
        # Residual blocks
        self.resblocks = nn.Sequential(
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256)
        )
        
        # Upsampling
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        # Output layer
        self.output_conv = nn.Sequential(
            nn.Conv2d(64, 3, kernel_size=7, padding=3),
            nn.Tanh()
        )
    
    def forward(self, x, mask):
        # Concatenate input image and mask
        x_input = torch.cat([x, mask], dim=1)
        
        # Encoder
        e0 = self.init_conv(x_input)
        e1 = self.down1(e0)
        e2 = self.down2(e1)
        
        # Residual blocks
        r = self.resblocks(e2)
        
        # Decoder
        d1 = self.up1(r)
        d2 = self.up2(d1)
        
        # Output
        output = self.output_conv(d2)
        
        # Create a smoothed mask for blending
        smoothed_mask = F.avg_pool2d(F.avg_pool2d(mask, 3, stride=1, padding=1), 3, stride=1, padding=1)
        
        # Combine original image and generated image using the mask
        # Original content outside mask, generated content inside mask
        result = x * (1 - smoothed_mask) + output * smoothed_mask
        
        return result

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        # Input: 3 channel image
        self.layers = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Layer 2
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Layer 3
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Layer 4
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Output
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )
    
    def forward(self, x):
        return self.layers(x)

# Combined loss function
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.l1_loss = nn.L1Loss()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, outputs, targets, masks, d_outputs=None):
        # Content loss (areas outside mask should remain unchanged)
        content_loss = self.l1_loss(outputs * (1 - masks), targets * (1 - masks)) * 10.0
        
        # Reflection area loss
        reflection_loss = self.l1_loss(outputs * masks, targets * masks) * 5.0
        
        # Edge consistency loss for smooth transitions
        # Create a boundary region around reflections
        dilated_mask = F.max_pool2d(masks, kernel_size=5, stride=1, padding=2)
        boundary_mask = dilated_mask - masks
        
        # Edge loss focuses on the boundary area
        edge_loss = self.l1_loss(outputs * boundary_mask, targets * boundary_mask) * 20.0
        
        # Adversarial loss (optional, used if discriminator is available)
        adv_loss = torch.tensor(0.0, device=outputs.device)
        if d_outputs is not None:
            adv_loss = -torch.mean(d_outputs) * 0.1  # WGAN-style adversarial loss
        
        # Total loss
        total_loss = content_loss + reflection_loss + edge_loss + adv_loss
        
        # Return loss components for logging
        loss_components = {
            'content': content_loss.item(),
            'reflection': reflection_loss.item(),
            'edge': edge_loss.item(),
            'adversarial': adv_loss.item() if isinstance(adv_loss, torch.Tensor) else adv_loss
        }
        
        return total_loss, loss_components

# Training function
def train_model(frame_data, epochs=50, batch_size=8, save_dir="results"):
    """Train the reflection removal model using GAN approach"""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Create test frames for visualization
    test_frames = random.sample(frame_data, min(5, len(frame_data)))
    
    # Create training dataset
    train_frames = random.sample(frame_data, min(500, len(frame_data)))
    dataset = ReflectionDataset(train_frames, patch_size=256, augment=True)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    # Create test dataset
    test_dataset = ReflectionDataset(test_frames, augment=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Initialize models
    generator = InpaintingGenerator().to(device)
    discriminator = Discriminator().to(device)
    
    # Initialize optimizers
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))
    
    # Learning rate scheduler
    g_scheduler = torch.optim.lr_scheduler.MultiStepLR(g_optimizer, milestones=[epochs//3, epochs*2//3], gamma=0.5)
    d_scheduler = torch.optim.lr_scheduler.MultiStepLR(d_optimizer, milestones=[epochs//3, epochs*2//3], gamma=0.5)
    
    # Loss function
    criterion = CombinedLoss()
    
    # Create directory for progress visualization
    progress_dir = os.path.join(save_dir, "progress")
    if not os.path.exists(progress_dir):
        os.makedirs(progress_dir)
    
    # Training loop
    best_loss = float('inf')
    start_time = time.time()
    
    for epoch in range(epochs):
        generator.train()
        discriminator.train()
        
        # Track losses
        g_losses = []
        d_losses = []
        loss_component_sums = {'content': 0, 'reflection': 0, 'edge': 0, 'adversarial': 0}
        
        print(f"Epoch {epoch+1}/{epochs}")
        progress_bar = tqdm(dataloader, desc="Training")
        
        for i, (images, masks) in enumerate(progress_bar):
            images, masks = images.to(device), masks.to(device)
            batch_size = images.size(0)
            
            # ---------------------
            # Train Discriminator
            # ---------------------
            if i % 3 == 0:  # Train D less frequently
                d_optimizer.zero_grad()
                
                # Real images
                real_outputs = discriminator(images)
                
                # Generate fake images
                with torch.no_grad():
                    fake_images = generator(images, masks)
                fake_outputs = discriminator(fake_images.detach())
                
                # Compute loss (WGAN)
                d_loss = torch.mean(fake_outputs) - torch.mean(real_outputs)
                
                # Add gradient penalty
                alpha = torch.rand(batch_size, 1, 1, 1, device=device)
                interpolates = alpha * images + (1 - alpha) * fake_images.detach()
                interpolates.requires_grad_(True)
                
                d_interpolates = discriminator(interpolates)
                
                gradients = torch.autograd.grad(
                    outputs=d_interpolates,
                    inputs=interpolates,
                    grad_outputs=torch.ones_like(d_interpolates),
                    create_graph=True,
                    retain_graph=True
                )[0]
                
                gradients = gradients.view(batch_size, -1)
                gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * 10
                
                d_loss = d_loss + gradient_penalty
                
                d_loss.backward()
                d_optimizer.step()
                
                d_losses.append(d_loss.item())
            
            # ---------------------
            # Train Generator
            # ---------------------
            g_optimizer.zero_grad()
            
            # Generate fake images
            fake_images = generator(images, masks)
            
            # Adversarial loss (only if discriminator is being used)
            fake_outputs = None
            if i % 3 == 0:
                fake_outputs = discriminator(fake_images)
            
            # Calculate loss
            g_loss, loss_components = criterion(fake_images, images, masks, fake_outputs)
            
            g_loss.backward()
            g_optimizer.step()
            
            # Track losses
            g_losses.append(g_loss.item())
            for k, v in loss_components.items():
                loss_component_sums[k] += v
            
            # Update progress bar
            progress_bar.set_postfix({
                'G_loss': f"{g_loss.item():.4f}",
                'D_loss': f"{d_losses[-1] if d_losses else 0:.4f}"
            })
        
        # Update learning rates
        g_scheduler.step()
        d_scheduler.step()
        
        # Calculate average losses
        avg_g_loss = sum(g_losses) / len(g_losses) if g_losses else 0
        avg_d_loss = sum(d_losses) / len(d_losses) if d_losses else 0
        avg_components = {k: v / len(g_losses) for k, v in loss_component_sums.items()} if g_losses else {}
        
        # Print stats
        elapsed = time.time() - start_time
        curr_g_lr = g_optimizer.param_groups[0]['lr']
        curr_d_lr = d_optimizer.param_groups[0]['lr']
        
        print(f"Epoch {epoch+1}/{epochs}, G Loss: {avg_g_loss:.4f}, D Loss: {avg_d_loss:.4f}, "
              f"Content: {avg_components.get('content', 0):.2f}, "
              f"Edge: {avg_components.get('edge', 0):.2f}, "
              f"Time: {elapsed:.1f}s, G LR: {curr_g_lr:.6f}")
        
        # Save best model
        if avg_g_loss < best_loss:
            best_loss = avg_g_loss
            torch.save({
                'generator': generator.state_dict(),
                'discriminator': discriminator.state_dict(),
                'epoch': epoch
            }, os.path.join(save_dir, "best_model.pth"))
            print(f"New best model saved with G loss: {best_loss:.4f}")
        
        # Save latest model
        torch.save({
            'generator': generator.state_dict(),
            'discriminator': discriminator.state_dict(),
            'epoch': epoch
        }, os.path.join(save_dir, "latest_model.pth"))
        
        # Visualization
        if epoch % 5 == 0 or epoch == epochs - 1:
            generator.eval()
            with torch.no_grad():
                for i, (test_image, test_mask) in enumerate(test_loader):
                    test_image = test_image.to(device)
                    test_mask = test_mask.to(device)
                    
                    # Generate output
                    test_output = generator(test_image, test_mask)
                    
                    # Convert to images for display
                    input_img = test_image[0].cpu().numpy().transpose(1, 2, 0) * 255
                    mask_img = test_mask[0, 0].cpu().numpy() * 255
                    output_img = test_output[0].cpu().numpy().transpose(1, 2, 0) * 255
                    
                    # Clip values
                    input_img = np.clip(input_img, 0, 255).astype(np.uint8)
                    mask_img = np.clip(mask_img, 0, 255).astype(np.uint8)
                    output_img = np.clip(output_img, 0, 255).astype(np.uint8)
                    
                    # Create visualization
                    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                    
                    axes[0].imshow(input_img)
                    axes[0].set_title("Input Image")
                    axes[0].axis("off")
                    
                    axes[1].imshow(mask_img, cmap='gray')
                    axes[1].set_title("Reflection Mask")
                    axes[1].axis("off")
                    
                    axes[2].imshow(output_img)
                    axes[2].set_title("Reflection Removed")
                    axes[2].axis("off")
                    
                    plt.tight_layout()
                    plt.savefig(os.path.join(progress_dir, f"sample_{i}_epoch_{epoch+1}.png"))
                    plt.close()
    
    return generator

# Process video using trained model
def process_video(generator, frame_data, output_dir="processed_frames", fps=30):
    """Process the entire video with the trained model"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create dataset for full video
    full_dataset = ReflectionDataset(frame_data, augment=False)
    
    generator.eval()
    with torch.no_grad():
        print("Processing video frames...")
        progress_bar = tqdm(enumerate(frame_data), total=len(frame_data))
        for i, (frame_path, original_size) in progress_bar:
            # Load original frame
            orig_frame = cv2.imread(frame_path)
            orig_height, orig_width = orig_frame.shape[:2]
            
            # Get processed frame from dataset
            frame_tensor, mask_tensor = full_dataset[i]
            
            # Skip processing if no reflections detected
            if torch.sum(mask_tensor) == 0:
                cv2.imwrite(f"{output_dir}/frame_{i:04d}.png", orig_frame)
                continue
            
            # Process the frame
            frame_tensor = frame_tensor.unsqueeze(0).to(device)
            mask_tensor = mask_tensor.unsqueeze(0).to(device)
            output = generator(frame_tensor, mask_tensor)
            
            # Convert output tensor to image
            output_img = output[0].cpu().numpy().transpose(1, 2, 0) * 255
            output_img = np.clip(output_img, 0, 255).astype(np.uint8)
            output_img = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)
            
            # Resize back to original dimensions
            output_img = cv2.resize(output_img, (orig_width, orig_height))
            
            # Save processed frame
            cv2.imwrite(f"{output_dir}/frame_{i:04d}.png", output_img)
    
    # Combine frames back into video
    print("Creating video from processed frames...")
    os.system(f"ffmpeg -framerate {fps} -i {output_dir}/frame_%04d.png -c:v libx264 -pix_fmt yuv420p -crf 18 processed_video.mp4")
    print("Processed video saved as processed_video.mp4")

# Main execution
if __name__ == "__main__":
    # Extract frames
    video_path = "video.mp4"
    frame_data, video_fps = extract_frames(video_path)
    
    # Train model
    generator = train_model(frame_data, epochs=50, batch_size=8, save_dir="results")
    
    # Process full video
    process_video(generator, frame_data, fps=video_fps)

Using device: cuda
Total frames: 1548, FPS: 30, Resolution: 1280x720
Epoch 1/50


Training: 100%|██████████| 63/63 [00:24<00:00,  2.58it/s, G_loss=0.0921, D_loss=168.3369] 


Epoch 1/50, G Loss: 0.1388, D Loss: 4169.8215, Content: 0.01, Edge: 0.02, Time: 24.4s, G LR: 0.000200
New best model saved with G loss: 0.1388
Epoch 2/50


Training: 100%|██████████| 63/63 [00:24<00:00,  2.60it/s, G_loss=0.0890, D_loss=54.4915] 


Epoch 2/50, G Loss: 0.0858, D Loss: 104.0735, Content: 0.01, Edge: 0.01, Time: 51.6s, G LR: 0.000200
New best model saved with G loss: 0.0858
Epoch 3/50


...

Training: 100%|██████████| 63/63 [00:24<00:00,  2.56it/s, G_loss=0.0184, D_loss=0.2074]


Epoch 50/50, G Loss: 0.0227, D Loss: 0.4266, Content: 0.00, Edge: 0.00, Time: 1256.6s, G LR: 0.000050
New best model saved with G loss: 0.0227
Processing video frames...


100%|██████████| 1548/1548 [03:12<00:00,  8.04it/s]


Creating video from processed frames...
Processed video saved as processed_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab